## Extract A280 and wavelength scan data from folder with lunatic reports

* Code assumes the lunatic report contains both "Concentration (mg/mL)" and wavelength scanning results.
* Please specify input and output folder paths before running the code.
* Search keywords need to be specified as well in order to index output files. 

## Prepare coding environment

In [3]:
import re
import os
import glob

import numpy as np
import pandas as pd

## Prepare filepaths
Specify input and output folder paths here. 

In [5]:
# Define the input folder path
folder_path = './example_data'
# Define the output folder path
output_filepath = "./output/"

## Create functions to rename files, clean and extract data

Specify search keywords here to index output files.

In [8]:
# Find keywords in file names
def rename_df (value):
    def get_exp_details(value):
        match = re.match(r"^.*(pH)(.*)$", value)
        if match:
            return "".join(match.groups())
        else:
            return "N/A"
    
    pH_plate = get_exp_details(value)
    dataframe_name = pH_plate.replace(" ", "").replace(".", "").replace("xlsx", "").replace("plate", "_plate")
    return dataframe_name

In [9]:
# Clean out metadata above "Plate ID" in column 1
def clean_data (file_path):
    df = pd.read_excel(file_path)
    index = df[df['Report'].astype(str).str.contains('Plate ID', na=False)].index[0] # Find the index of the row where 'Plate ID' appears in column 1
    filtered_df = df.loc[index:] # Filter out all rows above the found index
    
    # Use the first row as the header and drop it from the data
    filtered_df.columns = filtered_df.iloc[0]  # Set the first row as the header
    filtered_df = filtered_df.drop(index)          # Drop the first row now that it is used as the header
    clean_df = filtered_df.reset_index(drop=True) # Reset the index to start from 0
    return clean_df

In [10]:
# create a function to extract "Concentration (mg/mL)" from .csv files
def extract_con(df):
    # Extract columns 'Plate\nPosition' and 'Concentration\n(mg/ml)'
    reseted_df = df[['Plate\nPosition', 'Concentration\n(mg/ml)']]
    con_df = reseted_df.rename(columns={
        'Plate\nPosition': 'PlatePosition',
        'Concentration\n(mg/ml)': 'Concentration(mg/mL)',
    })
    
    pd.set_option('future.no_silent_downcasting', True) # future-proof the code related to how panda handles downcasting
    con_df = con_df.infer_objects(copy=False) # ensure the object columns are inferred correctly without silent downcasting
    
    con_df = con_df.fillna("N/A")
    
    return con_df

In [11]:
# create a function to extract wavelength scanning data from column 11 to column 231
def extract_scan (df):
    scan_df = df.iloc[:, 10:231]  # Use 10 because slicing in Python excludes the endpoint
    scan_df.columns = scan_df.columns.str.slice(1, 4) # Use the 2nd to 4th characters of each column name as the new header
    scan_df = scan_df.apply(pd.to_numeric, errors='coerce') # make the entire dataframe numeric
    scan_df = scan_df.fillna("N/A")

    # Extract column 2 (index 1)
    plate_position = df.iloc[:, 1]

    # Add column 2 to scan_df
    scan_df = pd.concat([plate_position, scan_df], axis=1)
    scan_df = scan_df.rename(columns={
        'Plate\nPosition': 'PlatePosition'
    })
    return scan_df

## Read files from folder, process and store data in an output folder

**Create a list of file names from folder**

In [14]:
# Use glob to find all Excel files in the folder
excel_files = glob.glob(os.path.join(folder_path, '*.xls')) + glob.glob(os.path.join(folder_path, '*.xlsx'))

# Print the file paths in the folder
for file in excel_files:
    print(file)

./example_data/2024-10-31_122352_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH 8.5 plate3.xlsx
./example_data/2024-10-31_122352_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH 8.5 plate2.xlsx
./example_data/2024-10-31_122352_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH 7.5 plate1.xlsx
./example_data/2024-10-31_125129_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH 6.5 plate3.xlsx
./example_data/2024-10-31_122352_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH 6.5 plate1.xlsx
./example_data/2024-10-31_125129_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH 6.5 plate2.xlsx
./example_data/2024-10-31_122352_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH 8.5 plate1.xlsx
./example_data/2024-10-31_122352_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH7.5_plate3.xlsx
./example_data/2024-10-31_122352_401285_DPD__Mumps_IL_PreFHN_AEX_screen1 pH7.5_plate2.xlsx


**Process each file and save the outputs to the specified output folder**

In [16]:
# Excute functions for each files in the folder 
for file in excel_files:
    df_name = rename_df (file)
    clean_df = clean_data (file)
    con_df = extract_con (clean_df)
    scan_df = extract_scan (clean_df)
    
    # Specify output file saving path
    scan_filepath = output_filepath + df_name + '(wavelength_scan).csv'
    con_filepath = output_filepath + df_name + '(concentration).csv'

    # Export data into .csv files
    con_df.to_csv(con_filepath, index=False)
    scan_df.to_csv(scan_filepath, index=False)

    print (df_name + ' data extraction successful!')

pH85_plate3 data extraction successful!
pH85_plate2 data extraction successful!
pH75_plate1 data extraction successful!
pH65_plate3 data extraction successful!
pH65_plate1 data extraction successful!
pH65_plate2 data extraction successful!
pH85_plate1 data extraction successful!
pH75__plate3 data extraction successful!
pH75__plate2 data extraction successful!
